In [ ]:
from wq_api import *

session = sign_in(username,password)

In [ ]:
# 参数,alpha表达式见后
setting = {
    'region': 'GLB', ###修改region以适应你的数据集
    'universe': 'MINVOL1M', ###修改universe以适应你的数据集
    'delay': 1,
    'decay': 3,
    'neutralization':'MARKET'} ###修改delay以适应你的数据集

dataset_id_ls = ['analyst9','analyst69','model109','model110']

# 只要vector还是只要matrix,目前只支持处理一种数据类型
vec_only = True

In [ ]:
# data is restored in {index}_refine_1.csv, where index is generated by order
import os

dir_path = './data'

if not os.path.exists(dir_path):
    os.makedirs(dir_path)

index = 0
while True:
    to_path = f'{dir_path}/{index}_data_1.csv'
    if not os.path.exists(to_path):
        break
    index += 1

print(f'Next available file path: {to_path}')

In [ ]:
datafields = []

for i in dataset_id_ls:
    datafields_df = get_datafields(session, dataset_id=i, region=setting['region'], universe=setting['universe'], delay=setting['delay'])
    if vec_only:
        datafields_df = datafields_df[datafields_df['type'] == "VECTOR"]['id']
    else:
        datafields_df = datafields_df[datafields_df['type'] == "MATRIX"]['id']
    
    # 使用 extend 而不是 append
    datafields.extend(datafields_df.values)
datafields

In [ ]:
alphas=[]
for datafield in datafields:
    alpha = "regression_neut(vector_neut(ts_rank(vec_max({})/close,120),ts_median(cap, 120) ),abs(ts_mean({},252)/ts_std_dev({},252)))" \
            .format(datafield,'returns','returns')
    alphas.append(alpha)
    
alphas = list(set(alphas))
alphas

In [ ]:
# make test alpha list
priority_alpha_list=[]
priority_alpha_list = [
    {
        'type': 'REGULAR',
        'settings': {
            'instrumentType': 'EQUITY',
            'region': 'GLB',
            'universe': setting['universe'],
            'delay': setting['delay'],
            'decay': setting['decay'],
            'neutralization': setting['neutralization'],
            'truncation': 0.08,
            'pasteurization': 'ON',
            'unitHandling': 'VERIFY',
            'nanHandling': 'ON',
            'language': 'FASTEXPR',
            'visualization': False,
        },
        'regular': alpha
    }
    for alpha in alphas
]

In [ ]:
import pandas as pd
# 将 priority_alpha_list 转换为 DataFrame
priority_alpha_df = pd.DataFrame(priority_alpha_list)

# 将 settings 列中的字典展开为单独的列
settings_df = priority_alpha_df['settings'].apply(pd.Series)

# 将展开的 settings 列与其他列合并
priority_alpha_df = pd.concat([priority_alpha_df.drop(columns=['settings']), settings_df], axis=1)

print(priority_alpha_df)

priority_alpha_df.to_csv(to_path)